In [1]:
from datetime import datetime
from tensorflow import keras
from src.data.load_data import train_X, train_y, validation_X, validation_y, test_X, test_y
from src.models.model import NextItemPredictor
from src.models.predict import predict, avg_precision
from src.models.train import train
%load_ext tensorboard

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/src/data/load_data.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  names=['user_id', 'movie_id', 'rating', 'timestamp'])


In [2]:
logdir = "../logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [3]:
model = NextItemPredictor()
model.compile()

In [4]:
train(
    model,
    train_X=train_X,
    train_y=train_y,
    validation_X=validation_X,
    validation_y=validation_y,
    batch_size=100,
    epochs=30,
    callbacks=[tensorboard_callback],
    description='batch100-epochs30'
)

Train on 4081 samples, validate on 1575 samples
Epoch 1/30
4081/4081 [==============================] - 23s 6ms/sample - loss: 8.0839 - val_loss: 7.6644
Epoch 2/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.3173 - val_loss: 7.7918
Epoch 3/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.1699 - val_loss: 7.8042
Epoch 4/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.1208 - val_loss: 7.8483
Epoch 5/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.0899 - val_loss: 7.9159
Epoch 6/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.0737 - val_loss: 7.9382
Epoch 7/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.0586 - val_loss: 7.9763
Epoch 8/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.0482 - val_loss: 7.9758
Epoch 9/30
4081/4081 [==============================] - 21s 5ms/sample - loss: 7.0392 - val_loss: 8.0416
Epoch 1

In [5]:
y_pred = predict(model, test_X)

In [6]:
avg_precision(test_y, y_pred)




/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


5.3696048149843e-06